#######################################################################
<br> ######  Made by: Dr. Keungoui Kim 
<br> ######  Title: KISDI Megatrend Project - Data Prep 05. Topic Modelling
<br> ######  goal : Megatrend Tech Analysis
<br> ######  Data set: KIPRIS
<br> ######  Time Span: 
<br> ######  Variables
<br> ######  Input: 
<br> ######  Output: 
<br> ######  Methodology: 
<br> ######  Time-stamp: 
<br> ######  Notice :
<br> #######################################################################

In [ ]:
# https://christinarok.github.io/2021/04/08/mecab.html

In [1]:
import os

os.getcwd()

'd:\\Google Drive(awekim@handong.edu)\\[Project]\\KISDI 과제\\02_메가트렌드'

In [2]:
import pandas as pd

data = pd.read_csv('Megatred_R/Abstract_trend.csv')
data = data.loc[~data['초록'].isna()]
data['type'] = data['type'].str.replace(': ', '-')
print(data.shape)
data.head()

(85086, 3)


,출원번호,type,초록
14,1019910000249,플랫폼의 전방위적 확산,본 발명은 사전이나 백과사전과 같은 텍스트데이터가 기록되는 데이터디스크와 그 데이터...
17,1019910015464,플랫폼의 전방위적 확산,"카드리더와 키보드를 이용하여 신용카드를 조회할 수 있는 시스템으로서, 통신포트를 선..."
18,1019920003044,플랫폼의 전방위적 확산,본 발명에서는 비교적 적은 수의 수용 가능 이미지를 합성하여 초기 판별함수를 형성함...
21,1019920010843,플랫폼의 전방위적 확산,어떤 상품을 선택할때 우리는 먼저 디자인과 색상을 살펴보고 이상적일때 다음으로 그제...
22,1019920012594,플랫폼의 전방위적 확산,"본 발명은 상품을 판매하는 방법에 관한 것으로, 특히 주둔매점 판매형식을 일반화하여..."


In [4]:
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from konlpy.tag import Kkma

# okt = Okt()
kkma = Kkma()

postposition_patterns = [
    r'을$', r'를$', r'이$', r'가$', r'은$', r'는$', r'의$', r'에$', r'에서$', r'로$', r'으로$', r'과$', r'와$', r'에 관한$', r'하다$', r'있다$', r'한다$', r'하는$', r'하여$'
]

stopwords = ['대한', '있다', '위한', '제1', '있습니다', '한다', '하는', '이것', '그것', '또는', '있음', '그리고', '하지만',
             '관한', '있', '통해','상기','제2','관','티','위','포함','단계','발명']

def preprocess_topics(topic_list):
    cleaned_topics = []
    for topic in topic_list:
        words = topic.split(', ')
        cleaned_words = []
        for word in words:
            for pattern in postposition_patterns:
                word = re.sub(pattern, '', word)
            if word not in stopwords and word and word not in cleaned_words:
                cleaned_words.append(word)
        cleaned_topics.append(cleaned_words)
    return cleaned_topics

def preprocess_text(text):
    words = text.split(', ')
    cleaned_words = []
    for word in words:
        pos_tags = kkma.pos(word)
        cleaned_words.extend([w for w, pos in pos_tags if pos in ['NNG', 'NNP'] and w not in stopwords])
        # cleaned_words.extend([w for w, pos in pos_tags if pos not in ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'EF', 'EC']])
    return list(set(cleaned_words))

from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

### https://github.com/jhgan00/ko-sentence-transformers
model = SentenceTransformer('jhgan/ko-sroberta-multitask') 
# MLP-KTLim/llama-3-Korean-Bllossom-8B, BM-K/KoSimCSE-roberta-multitask, xlm-r-bert-base-nli-stsb-mean-tokens
# sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens
# snunlp/KR-SBERT-V40K-klueNLI-augSTS

for i in list(data.type.unique()):
    temp = data.loc[data.type==i]
    documents = temp['초록']
    documents
    
    preprocessed_docs = [doc for doc in documents]
    
    vectorizer_model = CountVectorizer(ngram_range=(1, 2))

    topic_model = BERTopic(
            embedding_model=model,
            nr_topics=15,
            min_topic_size=int(data['출원번호'].nunique()*0.005),
            vectorizer_model=vectorizer_model,
            ctfidf_model=ctfidf_model,
            top_n_words = 20
        )

    topics, probs = topic_model.fit_transform(preprocessed_docs)
    # topics = topic_model.reduce_outliers(preprocessed_docs, topics)
    
    topic_info = topic_model.get_topic_info()

    topic_words = [topic_model.get_topic(topic) for topic in topic_info.Topic]
    topic_words_ed = [', '.join([word for word, _ in words]) for words in topic_words]
    
    # cleaned_topics = preprocess_topics(topic_words_ed)
    cleaned_topics = [preprocess_text(topic) for topic in topic_words_ed]
    
    # topic modelling results
    result_df = pd.DataFrame({
        'Topic': topic_info['Topic'],
        'Count': topic_info['Count'],
        'Name': topic_info['Name'],
        'Top Words': cleaned_topics #[', '.join([word for word, _ in words]) for words in topic_words]
    })
    result_df.to_csv('bertopic_results_'+str(i)+'.csv', index=False, encoding='utf-8-sig')
      
    # doc-topic label
    df_docs = pd.DataFrame({
    'Document': preprocessed_docs,
    'Topic': topics
    })
    df_docs.to_csv('doc_topics_'+str(i)+'.csv', index=False, encoding='utf-8-sig')
    
    print(i)
        

플랫폼의 전방위적 확산
자동화-노동 형태 다변화
초개인화-맞춤화
가상화-융합화
